In [ ]:
from __future__ import print_function
import socket
import argparse
import requests
import re
import pandas as pd
import numpy as np
import pickle

In [ ]:
# Special thanks to Justin Bollinger @bandrel for the script
# https://gist.github.com/bandrel/1573f0d7ef9d32e4cbf7639e35fcea79

#parser = argparse.ArgumentParser(description='Enumerate subdomains via certificate transparency logs')
#parser.add_argument('domain', type=str, help='Target domain name')
#parser.add_argument('-i', action='count', default=0, help='Enables printing of hosts believed to be internal only')
#parser.add_argument('-v', action='count', help='Enables verbose output, -v for verbose or -vv for very verbose', default=0)

#args = parser.parse_args()

#target_domain = args.domain
#verbose = args.v
internal_hosts = []
dfhosts = pd.DataFrame(columns = {'DomainName': [] , 'addr': []}) 

# Enter the target to be searched
target_domain = 'derbycon.com'
verbose = 0
internal = 1

r = requests.get('https://crt.sh/?q=%.'+target_domain)
hostnames = re.findall(r'<TD>(\S+\.'+target_domain+r')',r.text)
uniq_hostnames = set(hostnames)
if verbose:
    print('Domain names found:')
    for host in uniq_hostnames:
        print(host)

for host in uniq_hostnames:
    try:
        data = socket.gethostbyname_ex(host)
        print(data[0]+'\t'+data[2][0])
        
        # Custom
        dfhosts = dfhosts.append({'DomainName': data[0], 'addr': data[2][0]}, ignore_index=True)
    except socket.gaierror:
        if verbose:
            print('{host} does not resolve, may be resolveable internally only'.format(host=host))
        internal_hosts.append(host)


#if args.i:
if internal:
    print('\n\nHostnames exist in certificate transparancy logs, but do not resolve')
    for host in internal_hosts:
        print(host)

In [ ]:
dfhosts.to_pickle('dfhosts.pkl')  # where to save it, usually as a .pkl
print(dfhosts)

In [ ]:
# Load an external notebook with normalized functions
%run ./centralprocessing.ipynb
df_min = get_location(dfhosts, 'addr')
df_min.head(10)

In [ ]:
# Load an external notebook with normalized functions
%run ./centralprocessing.ipynb
df_plot = prepare_location(df_min)
df_plot.head(100)

In [ ]:
# Load an external notebook with normalized functions
%run ./centralprocessing.ipynb
fig = get_heatmap(df_plot)
fig